This notebook  demonstrates the use of new SHACL shapes for controlled vocabularies in a validation process. The SHACL shapes used in this demonstration conform to the [solution specification](https://github.com/OP-TED/model2owl/issues/228#issuecomment-2557009873) and the [model2owl documentation](https://meaningfy-ws.github.io/model2owl-docs-gh-pages). The notebook uses real ([EU vocabularies](https://op.europa.eu/en/web/eu-vocabularies/e-procurement/tables)) and crafted vocabularies to test both valid and invalid setups for both _permissive_ and _restrictive_ [constraint levels](https://meaningfy-ws.github.io/model2owl-docs-gh-pages/public-review/uml/conv-enumerations.html#rule:enumeration-constraint-level).

The generation of SHACL shapes using model2owl and testing them is beyond the scope of this notebook as this is already covered by model2owl test suite. However, the notebook relies on SHACL shapes in the same form as those generated by model2owl.

A minimal, crafted dataset of RDF resources is used, aligned with the [ePO ontology](https://github.com/OP-TED/ePO), to demonstrate the application of constraints implemented in the SHACL shapes.


In [87]:
# install required packages
! pip install pyshacl
! pip install rdflib

In [88]:
from pathlib import Path
from pyshacl import validate
import rdflib

In [89]:
CWD = Path().resolve()
TESTS_PATH = Path(CWD).parent.resolve()
DATA_DIR = TESTS_PATH / "testData" / "noticeData"
CV_DATA_DIR = TESTS_PATH / "testData" / "cv"
SHAPES_DATA_PATH = TESTS_PATH / "testData" / "shacl" / "cv-shapes.ttl"

**Note**: Set the variable below to _True_ to enable printing of extra information

In [90]:
VERBOSE = False

Define helper functions

In [91]:
def load(one_or_more_rdf_files: Path | tuple[Path] | list[Path]) -> rdflib.Graph:
    graph = rdflib.Graph()
    files = (
        [one_or_more_rdf_files] if isinstance(one_or_more_rdf_files, Path) else one_or_more_rdf_files
    )
    for f in files:
        graph.parse(f)
    return graph


def run_validation(data_graph: rdflib.Graph, shacl_shape_graph: rdflib.Graph):
    return validate(
        data_graph,
        shacl_graph=shacl_shape_graph,
        meta_shacl=False,
        js=False,
        debug=False,
    )

Define test cases for permissive and restrictive constraint levels together with the expected validation result

In [92]:
cases = (
    # (description, rdf_data, validation_result)
    (
        "01: permissive shape for hasReservedExecution with at-voc:applicability concept",
        ((DATA_DIR / "data01.ttl", CV_DATA_DIR / "applicability-skos.rdf"), SHAPES_DATA_PATH),
        True,
    ),
    (
        "02: permissive shape for hasReservedExecution with an arbitrary concept",
        ((DATA_DIR / "data02.ttl", CV_DATA_DIR / "dummy-concepts.ttl"), SHAPES_DATA_PATH),
        True,
    ),
    (
        "03: validation using restrictive shape for hasEInvoicingPermission with a concept belonging to at-voc:permission",
        ((DATA_DIR / "data03.ttl", CV_DATA_DIR / "permission-skos.rdf"), SHAPES_DATA_PATH),
        True,
    ),
    (
        "04: validation using restrictive shape for hasContractNatureType with a concept belonging to an arbitrary scheme fails",
        ((DATA_DIR / "data04.ttl", CV_DATA_DIR / "dummy-concepts.ttl"), SHAPES_DATA_PATH),
        False,
    ),
    (
        "05: validation using permissive shape for hasReservedExecution with an arbitrary resource fails",
        ((DATA_DIR / "data05.ttl", CV_DATA_DIR / "dummy-concepts.ttl"), SHAPES_DATA_PATH),
        False,
    ),
)


Run tests for the defined cases

In [93]:
for case_descr, input_data, expected in cases:
    if VERBOSE:
        print(f"*** case {case_descr} ***")
    data_files, shapes_files = input_data
    data_graph = load(data_files)
    shapes_graph = load(shapes_files)
    if VERBOSE:
        print("Data graph:")
        print(data_graph.serialize(format="ttl"))
        print("Shapes graph:")
        print(shapes_graph.serialize(format="ttl"))
    conforms, results_graph, results_text = run_validation(
        data_graph,
        shapes_graph,
    )
    if VERBOSE:
        print(results_text)
    assert conforms == expected, f"'{case_descr}' failed."